# Modelo Base para predicción 


In [27]:
#Importar librerías
import numpy as np 
import pandas as pd 
import joblib 
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

#cargar dataset
df = pd.read_csv('../data/pet_adoption_data.csv')

#separar X e y
y = df['TimeInShelterDays']
X = df.drop(columns=['PetID', 'TimeInShelterDays', 'AdoptionLikelihood'])

#Train/Test train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42
)

#Cargar Preprocesador serializado
preprocessor = joblib.load('../models/preprocessor.joblib')

In [28]:
#Funcion de Evaluación 
def evaluate_model(model, X_train, y_train, X_val, y_val):
    """Devuelve MAE, RMSE y R2 para train y validation"""
    results = {}
    
    for name, X_set, y_set in [('train', X_train, y_train), ('val', X_val, y_val)]:
        y_pred = model.predict(X_set)
        mae = mean_absolute_error(y_set, y_pred)
        rmse = np.sqrt(mean_squared_error(y_set, y_pred))
        r2 = r2_score(y_set, y_pred)
        results[name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
        print(f"{name} metrics:", results[name])
        
    return results

Evalúa un modelo para train y validación (test).

Calcula 3 métricas:
MAE → error absoluto promedio
RMSE → error cuadrático medio, penaliza errores grandes
R² → qué tan bien explica el modelo la variabilidad

Devuelve un diccionario con resultados por conjunto.

#### Entrenamiento de Modelos Baseline

In [29]:
#5.1 Linear Regression
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
lr_pipeline.fit(X_train, y_train)
lr_results = evaluate_model(lr_pipeline, X_train, y_train, X_test, y_test)


train metrics: {'MAE': 22.268411614117973, 'RMSE': np.float64(25.650293207132197), 'R2': 0.006826478417998771}
val metrics: {'MAE': 22.25941504857843, 'RMSE': np.float64(25.717864756974002), 'R2': -0.002079654263464459}


In [30]:
#5.2 Decision Tree Regresor
dt_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])
dt_pipeline.fit(X_train, y_train)
dt_results = evaluate_model(dt_pipeline, X_train, y_train, X_test, y_test)

train metrics: {'MAE': 0.0, 'RMSE': np.float64(0.0), 'R2': 1.0}
val metrics: {'MAE': 30.497512437810947, 'RMSE': np.float64(37.6275938759778), 'R2': -1.1450890406826755}


#### Comparación y Check de Overfitting

In [31]:
def check_overfitting(train_metrics, val_metrics, threshold=0.05):
    """Retorna True si la diferencia de R^2 entre train y val es menor a threshold"""
    diff = abs(train_metrics['R2']- val_metrics['R2'])
    return diff <= threshold, diff

#Linear Regression
lr_ok, lr_diff = check_overfitting(lr_results['train'], lr_results['val'])

#Decision Tree
dt_ok, dt_diff = check_overfitting(dt_results['train'], dt_results['val'])

print("Linear Regression:", lr_results, "Overfitting OK:", lr_ok, f"Diff={lr_diff:.3f}")
print("Decision Tree:", dt_results, "Overfitting OK:", dt_ok, f"Diff={dt_diff:.3f}")

Linear Regression: {'train': {'MAE': 22.268411614117973, 'RMSE': np.float64(25.650293207132197), 'R2': 0.006826478417998771}, 'val': {'MAE': 22.25941504857843, 'RMSE': np.float64(25.717864756974002), 'R2': -0.002079654263464459}} Overfitting OK: True Diff=0.009
Decision Tree: {'train': {'MAE': 0.0, 'RMSE': np.float64(0.0), 'R2': 1.0}, 'val': {'MAE': 30.497512437810947, 'RMSE': np.float64(37.6275938759778), 'R2': -1.1450890406826755}} Overfitting OK: False Diff=2.145


#### Selección y Serialización del mejor modelo
Elegimos el modelo con menor RMSE en validación que cumpla con el criterio de overfitting (diff <= 0.05).

In [32]:
#Evaluar candidatos(modelos)
candidates = []
if lr_ok:
    candidates.append(('LinearRegression', lr_pipeline, lr_results['val']['RMSE']))
if dt_ok:
    candidates.append(('DecisionTree', dt_pipeline, dt_results['val']['RMSE']))

#Seleccionar el mejor modelo
if candidates:
    best_name, best_model, best_rmse = min(candidates, key=lambda x: x[2])
    print(f"Mejor modelo: {best_name} con RMSE={best_rmse:.2f}")

    #Serializar
    joblib.dump(best_model, '../models/baseline_best_model.joblib')
    print("Mejor modelo serializado en '../models/baseline_best_model.joblib'")
else:
    print("Ningún modelo cumple el criterio de overfitting <5%")

Mejor modelo: LinearRegression con RMSE=25.72
Mejor modelo serializado en '../models/baseline_best_model.joblib'


## Conclusión del Modelo Base

- Se construyó un **modelo base (baseline)** para predecir `TimeInShelterDays` usando **Linear Regression** y **Decision Tree Regressor**.  
- Se utilizó un **pipeline de preprocesamiento** que transforma automáticamente las variables categóricas, numéricas y binarias, asegurando consistencia entre entrenamiento y prueba.  
- Ambos modelos fueron entrenados y evaluados en conjuntos de **train** y **test**, calculando las métricas **MAE, RMSE y R²**.  
- Se verificó el **overfitting** comparando el R² de train y test, con un límite del 5%.  
- Entre los modelos que cumplen el criterio de overfitting, se seleccionó el **mejor basado en menor RMSE en validación**.  
- En este caso, el **Linear Regression** resultó ser el mejor modelo con RMSE = 25.72 y fue **serializado** para uso posterior en producción (`baseline_best_model.joblib`).  

**Resumen:**  
El pipeline garantiza un preprocesamiento consistente, las métricas permiten evaluar desempeño y overfitting, y la selección final asegura que el modelo elegido sea confiable y generalizable a nuevos datos.
